In [ ]:
blender_version = '5.0.1' #@param ['5.0.1', '5.1.0'] {allow-input: true}
url_blend = '' #@param {type: 'string'}
#@markdown ---
animation = False #@param {type: 'boolean'}
start_frame =  1#@param {type: 'integer'}
end_frame =  250#@param {type: 'integer'}
#@markdown ---
output_name = 'blender-##' #@param {type: 'string'}
#@markdown Ruta en Drive compartido (ejemplo: "Blender Renders/Project1")
drive_output_path = 'Blender Renders' #@param {type: 'string'}
#@markdown ---
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = False #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}

In [ ]:
%cd /content

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4

os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.5.9"

In [ ]:
import shutil
import os
import re
from google.colab import files, drive

# Mount Google Drive
drive.mount('/drive')

# Verificar que la carpeta compartida exista
shared_folder_path = f'/drive/MyDrive/{drive_output_path}'
if not os.path.exists(shared_folder_path):
    # Intentar buscar en "Compartidos conmigo"
    shared_with_me = f'/drive/Shareddrives/{drive_output_path}'
    if os.path.exists(shared_with_me):
        print(f"Usando carpeta en Shared Drives: {shared_with_me}")
    else:
        print(f"⚠️ ADVERTENCIA: No se encontró la carpeta '{drive_output_path}'")
        print("Asegúrate de:")
        print("1. La carpeta esté compartida con esta cuenta")
        print("2. La ruta sea correcta")
        print("3. Hayas agregado acceso directo a 'Mi unidad' si está en 'Compartidos conmigo'")

# Descargar archivo .blend desde URL o Google Drive
uploaded_filename = os.path.basename(url_blend)

# Si es un link de Google Drive, extraer el ID
if 'drive.google.com' in url_blend:
    match = re.search(r'/d/([a-zA-Z0-9-_]+)', url_blend)
    if match:
        file_id = match.group(1)
        # Construir URL de descarga directa
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
        print(f"Detectado link de Google Drive. ID: {file_id}")
    else:
        print("❌ No se pudo extraer el ID del archivo de Google Drive")
        raise SystemExit("URL de Google Drive inválida")
else:
    download_url = url_blend
    print(f"Usando URL directa: {download_url}")

# Descargar archivo solo si no existe
if not os.path.exists(uploaded_filename):
    print(f"Descargando {uploaded_filename}...")
    !gdown $download_url -O $uploaded_filename
    if os.path.exists(uploaded_filename):
        print(f"✅ {uploaded_filename} descargado correctamente")
    else:
        print(f"❌ Error al descargar {uploaded_filename}")
        raise SystemExit("Fallo en la descarga del archivo")
else:
    print(f"✅ Archivo {uploaded_filename} ya existe, omitiendo descarga.")

In [ ]:
# Crear directorios solo si no existen
if os.path.exists('render'):
    !rm -rf render/*
else:
    !mkdir render

if not os.path.exists('trash'):
    !mkdir trash

if uploaded_filename.lower().endswith('.blend'):
    shutil.copy(uploaded_filename, 'render/')
    blend_file_path = uploaded_filename
else:
    raise SystemExit("Invalid file extension, only .blend files can be uploaded.")

In [ ]:
import requests

# Verificar si Blender ya está instalado
if not os.path.exists(blender_version):
    major_minor = ".".join(blender_version.split('.')[:2])
    blender_url = f"https://ftp.nluug.nl/pub/graphics/blender/release/Blender{major_minor}/blender-{blender_version}-linux-x64.tar.xz"
    base_url = os.path.basename(blender_url)
    
    try:
        response = requests.head(blender_url, allow_redirects=True, timeout=10)
        if response.status_code != 200:
            print(f"Download failed for version '{blender_version}'.")
            print("Error downloading: You may need to define the download archive manually above.")
        else:
            print(f"Download URL: {blender_url}")
            print(f"Base filename: {base_url}")
    except Exception as e:
        print(f"Error checking URL: {e}")
        print("Error downloading: You may need to define the download archive manually above.")
    
    !mkdir $blender_version
    !wget -nc $blender_url
    !tar -xkf $base_url -C ./$blender_version --strip-components=1
    print(f"Blender {blender_version} instalado correctamente.")
else:
    print(f"Blender {blender_version} ya está instalado.")

In [ ]:
# Enable GPU rendering (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, the current GPU (e.g. Tesla K80) is not supported and you need to switch back to CUDA.")
    renderer = "OPTIX"

In [ ]:
import glob

%cd /content

# Crear output solo si no existe
if os.path.exists('output'):
    for f in glob.glob('/content/output/*'):
        os.remove(f)
else:
    !mkdir output

if not drive_output_path.endswith('/'):
    drive_output_path += '/'

# Determinar ruta de Drive (MyDrive o Shared Drives)
drive_path = f'/drive/MyDrive/{drive_output_path}'
if not os.path.exists(drive_path):
    drive_path = f'/drive/Shareddrives/{drive_output_path}'

# Crear carpeta de destino si no existe
os.makedirs(drive_path, exist_ok=True)

# Rutas optimizadas - Output de Blender por defecto va a trash (se descarta)
output_path = '/content/trash/' + output_name
blender_exe = f'/content/{blender_version}/{blender_version}/blender'
blend_file = f'/content/render/{blend_file_path}'
setgpu_script = '/content/setgpu.py'
trash_path = '/content/trash'
output_folder = '/content/output'

print(f"📁 Guardando renders en: {drive_path}")
print(f"📤 Nodo File Output debe apuntar a: /content/output/")
print(f"🗑️ Output por defecto de Blender irá a: {trash_path} (se descarta)")

if animation:
    for frame in range(start_frame, end_frame + 1):
        # Limpiar output antes de renderizar
        for f in glob.glob('/content/output/*'):
            os.remove(f)
        
        # Limpiar trash antes de renderizar
        for f in glob.glob('/content/trash/*'):
            if os.path.isfile(f):
                os.remove(f)
        
        # Renderizar frame (output por defecto va a trash, File Output nodes van a /content/output/)
        !$blender_exe -b $blend_file -P $setgpu_script -E CYCLES -o '{output_path}' -noaudio -f $frame -- --cycles-device "{renderer}"
        
        # Copiar SOLO archivos de /content/output/ a Drive (los del nodo File Output)
        rendered_files = glob.glob('/content/output/*')
        if rendered_files:
            for file_path in rendered_files:
                if os.path.isfile(file_path):
                    filename = os.path.basename(file_path)
                    shutil.copy2(file_path, drive_path + filename)
                    print(f"✅ Archivo guardado: {drive_output_path}{filename}")
            print(f"✅ Frame {frame} completado - {len(rendered_files)} archivo(s) guardado(s)")
        else:
            print(f"⚠️ Advertencia: Frame {frame} no generó archivos en /content/output/")
        
        print("🗑️ Carpetas limpiadas para siguiente frame\n")
else:
    # Limpiar output antes de renderizar
    for f in glob.glob('/content/output/*'):
        os.remove(f)
    
    # Limpiar trash antes de renderizar
    for f in glob.glob('/content/trash/*'):
        if os.path.isfile(f):
            os.remove(f)
    
    # Un solo frame
    !$blender_exe -b $blend_file -P $setgpu_script -E CYCLES -o '{output_path}' -noaudio -f $start_frame -- --cycles-device "{renderer}"
    
    # Copiar SOLO archivos de /content/output/ a Drive
    rendered_files = glob.glob('/content/output/*')
    if rendered_files:
        for file_path in rendered_files:
            if os.path.isfile(file_path):
                filename = os.path.basename(file_path)
                shutil.copy2(file_path, drive_path + filename)
                print(f"✅ Archivo guardado: {drive_output_path}{filename}")
        print(f"✅ Renderizado completado - {len(rendered_files)} archivo(s) guardado(s)")
    else:
        print("⚠️ Advertencia: No se generaron archivos en /content/output/")
    
    print("🗑️ Carpetas limpiadas")

In [ ]:
print("Renderización completada exitosamente.")

## Disclaimer
Google Colab is targeted to researchers and students to run AI/ML tasks, data analysis and education, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies time by time. [Colab Pro and Colab Pro+](https://colab.research.google.com/signup) are available for those who wanted to have more powerful GPU and longer runtimes for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info. In some cases, it might be faster to use an online Blender renderfarm.

## License